### 解讀NMEA 0183

In [ ]:
import serial

port = serial.Serial(port='COM5', baudrate=9600)
print(port)


In [ ]:
port.close()

In [ ]:
print(logs)

In [1]:
# load GPS logs
with open('gps_log2.txt','r') as f:
  logs = f.read()

In [ ]:
lines = logs.split()
lines[0:5]

In [ ]:
for line in lines:
    fields = line.split(',')
    # 逐筆檢查GGA
    if fields[0] == '$GPGGA':
        # 是GGA就讀欄位
        if int(fields[6]) > 0:
            # 是 GGA 且 Position Fix Indicator > 0 就讀欄位
            # lat_s = fields[2]
            # lat_dd = int(lat_s[0:2])
            # lat_mm = int(lat_s[2:4])
            # lat_ss = float(lat_s[4:])*60
            # print(f'Latitude: {lat_dd}° {lat_mm}\' {lat_ss:.2f}"')
            # lon_s = fields[4]
            # lon_dd = int(lon_s[0:3])
            # lon_mm = int(lon_s[3:5])
            # lon_ss = float(lon_s[5:])*60
            # print(f"Longitude: {lon_dd}° {lon_mm}' {lon_ss:.2f}\"")
            print(line)
            print(fields[14])


In [24]:
s = '1.834'
f = float(s)
a = int(f)
b = f-a
b


0.8340000000000001

In [61]:
s='$GPGGA,065325.00,2452.51142,N,12115.97109,E,2,08,0.97,203.6,M,16.7,M,,0000*5B'
a = s.index('$')
b = s.index('*')
res = 0
for c in s[a+1:b]:
  res = ord(c) ^ res

print(f'{res:02X}')

5B


In [58]:
ord('b')-ord('a')


1

In [76]:
parseGGA('$GPGGA,065325.00,2452.51142,N,12115.97109,E,2,08,0.97,203.6,M,16.7,M,,0000*5B')


{'UTC': '06:53:25.00',
 'Latitude': 24.875190333333332,
 'N/S': 'N',
 'Longitude': 121.26618483333333,
 'E/W': 'E',
 'Fix': 'DGPS',
 'Satellites': 8,
 'HDOP': 0.97,
 'Altitude': 203.6,
 'Alt Unit': 'M',
 'Geoid Separation': 16.7,
 'GS Unit': 'M',
 'Checksum': '0000*5B'}

In [71]:
def checkSum(msg):
    try:
        data = msg[msg.index('$')+1:msg.index('*')]
        cs = 0
        for c in data:
            cs = cs ^ ord(c)
        return f'{cs:02X}' == msg[msg.index('*')+1:]
    except:
        return False


In [75]:
def parseGGA(data):
    msgs = data.split()
    ggas = list(filter(lambda m: 'GGA' in m, msgs))
    if len(ggas) <= 0:
        return None
    else:
        if not checkSum(ggas[-1]):
            return None
        values = ggas[-1].split(',')
        gga = dict()
        gga['UTC'] = f'{values[1][0:2]}:{values[1][2:4]}:{values[1][4:]}'
        gga['Latitude'] = (float(values[2][0:2]) + (float(values[2][2:])/60)
                           ) * (1 if values[3].upper() == 'N' else -1)
        gga['N/S'] = values[3]
        gga['Longitude'] = (float(values[4][0:3]) + (float(values[4][3:])/60)
                            ) * (1 if values[5].upper() == 'E' else -1)
        gga['E/W'] = values[5]
        gga['Fix'] = ['invalid', 'valid', 'DGPS', 'PPS', 'RTK',
                      'not supported', 'estimated', 'Manual', 'Simulation'][int(values[6])]
        gga['Satellites'] = int(values[7])
        gga['HDOP'] = float(values[8])
        gga['Altitude'] = float(values[9])
        gga['Alt Unit'] = values[10]
        gga['Geoid Separation'] = float(values[11])
        gga['GS Unit'] = values[12]
        gga['Checksum'] = values[14]
        return gga


In [6]:
buffer = logs
gga = parseGGA(buffer)
if gga:
  print(gga)

{'UTC': '06:42:08.00', 'Latitude': 24.876094333333334, 'N/S': 'N', 'Longitude': 13.933501666666666, 'E/W': 'E', 'Fix': 'valid', 'Satellites': 5, 'HDOP': 1.74, 'Altitude': 221.3, 'Alt Unit': 'M', 'Geoid Separation': 16.7, 'GS Unit': 'M', 'Checksum': '*53'}


In [ ]:
buffer = ''
while True:
  data = port.read_until()
  buffer += data.decode('ascii')
  gga = parseGGA(buffer)
  if gga:
    print(gga)